### SQL Server'da Yedekleme Yönetimi

* SQL Server'ı küçük işletmelerden büyük işletmelere kadar bir çok firma kullanır.
* İşletmeler büyüdükçe verinin önemi daha çok ortaya çıkar ve veri kaybının minimumda tutulması gerekir. 

# 1. Yedeklemeye Giriş

## Backup Alma Yöntemleri

### FULL BACKUP

* Standart backup yöntemidir.
* Tüm veritabanının yedeğinin alınması işlemidir.
* Aynı sürüm veya daha üst sürüm SQL Serverlarda çalışabilirdir. 
* Sistem çalışırken online olarak alınabilir.
* Sıkıştırma parametresi ile %95'lere kadar sıkıştırılarak alınabilir.
* Sadece backup alınan zamana dönülebilir.

### DIFFERENTIAL BACKUP

* Alınan son full backup ile differential backup alınan zaman arasındaki değişim farkının alındığı backup türüdür.
* Full backup'a göre daha az yer kaplar.
* Sistem çalışırken online olarak alınabilir.
* Sıkıştırma parametresi ile %95'lere kadar sıkıştırılarak alınabilir.
* Sadece backup alınan zamana dönülebilir.
    * Önce full backupa sonra differential backupa dönülür.

### TRANSACTION LOG BACKUP

* Transaction log dosyasının yedeklenmesidir.
* En son alınan backup hangisi ise (full, differential ya da transaction log farketmez) onunla arasındaki değişimi alır.
* Sistem çalışırken online olarak alınabilir.
* Sıkıştırma parametresi ile %95'lere kadar sıkıştırılarak alınabilir.
* Saniyelik olarak istediğimiz ana dönme imkanı verir.
* Bankalarda sıklıkla kullanılır.



# 2. Backup / Restore Uygulama

* Sistemden üzerinden nasıl backup alınıyor, nasıl restore ediliyor ve backup alma teknikleri nasıl çalışıyor?

### Hazırlık

* **BACKUPTEST** ismiyle bir database oluştur.

* İçerisine **DATES** adını verdiğin şöyle bir tablo ekle;

<img src = "https://i.hizliresim.com/93fqh4v.png">

* Bir de öylesine veri gir;

<img src = "https://i.hizliresim.com/lfnl16r.png">

## Backup

<img src = "https://i.hizliresim.com/sf2arz7.png">

<img src = "https://i.hizliresim.com/ahsc5qn.png">

<img src = "https://i.hizliresim.com/rnrxols.png">

## Restore

<img src = "https://i.hizliresim.com/c1mtdd3.png">

<img src = "https://i.hizliresim.com/t79dqwf.png">

* Restore ettikten sonra verilerle oynamaya var olan veritabanı üzerinden devam edebilirsin.

<img src = "https://i.hizliresim.com/sl5d8jq.png">

* Eğer veritabanını silip veya bir şekilde kaybedip tekrar restore edersen önceki backup aldığın haliyle geri döner.

<img src = "https://i.hizliresim.com/2imrsy2.png">

* Yedeklerin sıklığı ve risk analizi çok önemlidir.



# 3. Differential Backup Uygulama 1

* **DATES** tablosuna 1 saniye aralıklarla anlık olarak saat bilgisini atalım.

<img src = "https://i.hizliresim.com/ju1tgr6.png">

* <code>WAITFOR DELAY '00:00:01'</code> yapılacak işlemleri 1 saniye geciktirir. Gecikmeleri istediğin gibi elinle ayarlayabilirsin.

* Döngü koşulunu 500 yapıp, sorguyu çalıştırıp, yedek dosyayı sildikten sonra tekrar backupın scriptine ulaş ve bir full backup al. (FULL1)

<img src = "https://i.hizliresim.com/3ie14hd.png">

* Bu noktada bir çok veri veritabanına atılmış durumda ve işler haldedir.

<img src = "https://i.hizliresim.com/qxzlk8m.png">

* Tekrar bir backup al ancak bu sefer differential yaparak al. (DIFF1)

<img src = "https://i.hizliresim.com/g222me6.png">

* Döngü devam ederken aralıklarla 2 kere daha differential backup al. (DIFF2 ve DIFF3)
* Son olarak döngüyu durdurup bir differential backup daha al. (DIFF 4)

<img src = "https://i.hizliresim.com/9i8pyj9.png">

* Son halinde full backupın boyutu differential backuplardan fazladır ve differential backuplar sadece aradaki farkı alır.
* Bu noktada full backup dosyası ile en son full backup aldığın hale geri dönebilirsin.
* Eğer differential backuplardan birine dönmek istiyorsan tek başına differential backup dosyası yeterli olmaz, full backup dosyasıyla beraber dönmen gerekir.
* Differential backupları ne kadar sıklıkla alırsan onları dönmek istediğin tarihe göre istediğin tarihte dönebilirsin.
* Sen kaç differential backup dosyası eklersen ekle sadece tarihe göre en son aldığın differential backup gözükür.

# 4. Differential Backup Uygulama 2

* Databasei biraz şişirmek amaçlı içerisine 100000 kayıt atalım.

<img src = "https://i.hizliresim.com/hctt1ek.png">

### Tabloyu kopyalamak

* <code>SELECT * INTO</code> komutu ile elinde var olan bir tabloyu verileriyle çoğaltabilirsin.

<img src = "https://i.hizliresim.com/acf3pl3.png">

* Bu şekilde kopyalamalara devam ederek veritabanımızı biraz şişirelim.

<img src = "https://i.hizliresim.com/4wtpsv2.png">

* Ardından bu veritabanımızı full backup ile sıkıştırmadan yedeklersek şu şekilde bir boyutu olur.

<img src = "https://i.hizliresim.com/r018egd.png">

* Backup aldıktan sonra bir tablo daha kopyalayıp differential backup al. (Bir tablonun kapladığı alanı şekilde görebilirsin)

<img src = "https://i.hizliresim.com/26pit87.png">

* Bir differential backup aldıktan sonra aldığın tüm differential backuplar hiçbir değişiklik yapmasan bile en az önceki differential backup kadar olur. Ve değişiklik yapmaya başladığın takdirde zamanla differential backuplar fullbackup boyutunu geçebilir. Bu noktada tekrar full backup alman gerekir. 

<img src = "https://i.hizliresim.com/dmmucf7.png">

* Zaman zaman tablolar ekleyip yarım saatte bir differential backup aldığını düşünürsek zamanla backup dosyalarının kapladığı alan büyür ve toplamında full backupı geçmeye başlar.

<img src = "https://i.hizliresim.com/71s5zup.png">

**HER BİR DIFFERENTIAL BACKUP, SON ALINAN FULL BACKUP İLE O ANIN FARKINI ALIR. BUNU UNUTMA!**

# 5. Transaction Log Backup

* Full backup ve differential backup aksine veri dosyası üzerinde değil log dosyası üzerinde yedek alır.
* OLTP sistemlerde yapılan tüm hareketler loga yazıldığı için, logun yedeğini aldığında oradaki tüm hareketleri görebildiğin için aldığın yedek içerisindeki istediğin saniyeye, istediğin harekete doğrudan dönebilirsin.

<img src = "https://i.hizliresim.com/5o9lurh.png">

* Recovery model: **Full** seçildiği zaman transaction log dosyasına yazılıp ardından MDFe kaydedilen işlemler log dosyasından silinmeden korunmaya devam eder. Zamanla log dosyası data dosyasından daha büyük hale gelebilir ancak log dosyasından da her türlü ana geri dönüş sağlanılabilir. 
* Recovery model: **Bulk-logged** seçildiği zaman sadece yapılan toplu işlemler loglanır. Onun dışındaki tek satırlık basit sql cümleleri loglanmaz. 
* Recovery model: **Simple** seçildiği zaman hiçbir şey loglanmaz. İşlem bittikten sonra logdan otomatik temizlenir. Veritabanı performanslı çalışsın istiyorsan seçebilirsin ancak bu durumda da transaction log backup alamazsın. Transaction log backup alabilmen için full yapman gerekir.

* Şu anda mesela üzerinde çalıştığımız databasein log dosyası data dosyasından daha büyüktür.

<img src = "https://i.hizliresim.com/r27g6uk.png">

* Tablomuz içerisine 100 ms aralıklarla kayıt atmaya devam edelim.

<img src = "https://i.hizliresim.com/ciwwj7g.png">

* Bu noktada full backup dosyan sıkıştırmasız bu şekilde gözükür.

<img src = "https://i.hizliresim.com/qaeepz2.png">

* Transaction log backup bu script ile alabilirsin.

<img src = "https://i.hizliresim.com/4x6ng8i.png">

* Daha sonra aldığın tüm log backuplar kendisinden bir önce alınan backup ile farkını alır. Log, differential veya full fark etmez.

<img src = "https://i.hizliresim.com/szn3l43.png">

### Hata sonrası backupdan bir zamana dönme

* Diyelim sistem üzerinde çalışırken bir hata yaptın.

<img src = "https://i.hizliresim.com/rma6cm2.png">

* Son full backup ile de aranda yaklaşık 3000 kayıtlık bir fark var.

<img src = "https://i.hizliresim.com/39qb5u6.png">

* Bu durumda restore işlemi gerçekleştirmek gerekir. Hemen bir transaction log backup al.

<img src = "https://i.hizliresim.com/q9l8lkz.png">

* Restore ederken timelinedan dönmek istediğin spesifik bir zaman seçebilirsin. Ona göre ihtiyacın olan transaction log dosyaları seçilir.

<img src = "https://i.hizliresim.com/on14p5e.png">

* Şu ayarları da yapmayı ihmal etme.

<img src = "https://i.hizliresim.com/b425xfp.png">

#### Transaction log backup dezavantajı
* Database full modda olmalıdır.
    * Çok fazla okuma yazma işlemi gerçekleşir.
    * Hızlı diskler, performanslı sunucu gerektirir.
    * Her şeyi içinde barındırdığı için büyür ve kontrol etmesi zorlaşır.
        * 10 GBlik data dosyası 500 GBlık log dosyası bulunabilir.
* **Data kritikse, kaybolması tahammül edilemezse mecbur kullanacaksın.**

# 6. Database Attach/Detach İşlemi

* Databasei bir yerden başka bir yere backup/restore ile taşıyabilirsin.
* Peki bu dosyalar başka bir veritabanı sunucuya taşınsa çalışmaz mı?
* **Elinde başka bir sunucudan aldığın .MDF ve .LDF dosyaların varsa bunu yeni bir sunucuya atman o veritabanını görmeni sağlamaz.**
* Ancak onu buraya tanıtma yöntemi vardır.

<img src = "https://i.hizliresim.com/azfugyf.png">

* **Attach** sekmesinden .MDF dosyasını seçerek veritabanını bu sisteme tanıtabilirsin.
* Peki madem .MDF dosyasını böyle tanıtıp alabiliyoruz. Bu dosyaları başka bir yere taşıyıp orada da açsak ya?
    * SQL Server çalışırken bu databasein dosyalarını silemezsin, taşıyamazsın, kopyalayamazsın.
* Peki illa backup alıp kopyalayıp başka bir sunucuda restore etmem mi gerekiyor?
    * Hayır. **Detach** ederek .MDF ve .LDF dosyalarını kopyalanabilir hale getirebilirsin.
    
<img src = "https://i.hizliresim.com/j8hrgn5.png">

* Farklı bir veritabanı taşıma yöntemidir.
* Diğer bir yöntemde de **Take Offline** ile taşıyabilir. Ardından **Bring Online** diyebilirsin.

<img src = "https://i.hizliresim.com/hizos3j.png">

# 7. Backup Planlama Stratejisi

* Amacımız verinin minimum kayıpla çalışmasıdır. Bunun değerini ölçmek için de verinin bizim için ne kadar kıymetli olduğunu bilmemiz gerekir.
* Örneğin bir kargo şirketi için saniyelik bir veri bile çok kritik iken bazı şirketler için yarım günlük verinin kaybı pek önemsenmez.
* Eğer bir kargo şirketi veya bir alışveriş sitesi için veritabanı kulanacaksan kesinlikle **transaction log backup** kullanman gerekir. Günde 3-4 kez full backup, yarım saatte bir de transaction log backup alabilirsin. Transaction log dosyalarının büyümesine göre bu süreyi 10 dakikaya 5 dakikaya da düşürebilirsin. Bu süreler risk için değildir, dosyaların boyutları içindir. Çünkü zaten 5 dakika da olsa 30 dakika da olsa istediğin saniyeye dönebilirsin. Dezavantajı da sistemde fazladan yer kaplaması ve recovery model full'de olduğu için çok fazla işlem gerçekleştirmesi ve hızlı bir sistem gerektirmesidir.

### Eğer data kritik değilse;

* Diyelim günde bir kez gece 00.00'da 100 GBlik veritabanından full backup alıyorsun ve bu veritabanı yarım saatte 1 GB büyüyor. Bu durumda gün sonunda 148 GBlik bir verin olur. Bunun sıkıntısı şudur; gece 23.30da bir problem çıksa en yakın yedeğin yaklaşık 24 saat önce ve bu da 24 saat riski olan bir kayıp. 

<img src = "https://i.hizliresim.com/g3v77kb.png">

* Çok fazla tavsiye **edilmez!**

--------

* Yine aynı büyümeye sahip veri gün içerisinde 6 saat aralıklarla 4 kez full backup alıyor. Burada 6 saatlik bir risk söz konusudur, bazı şirketler bunu tolere edebilir, bazı şirketler çok önemlidir.

<img src = "https://i.hizliresim.com/r7425sc.png">

* Tercih **edilmeyebilir!**

---------

* Her zaman full backup almak zorunda değilsin. Yine gün içerisinde 4 full backup alsan, onun dışında da yarım saatte bir differential backup alsan. Risk yarım saate iner.

<img src = "https://i.hizliresim.com/4xmwbqj.png">


### Standart Backup Stratejisi

**Günde 4 kez fullbackup ve yarım saatte bir differential backup'dan oluşur**

* Sadece günde 4 kez full backup alınıp 6 saat riskle bu kadar boyuta sahipken;

<img src = "https://i.hizliresim.com/dlhnn9e.png">

* Full backup üstüne yarım saatte bir differential backup alırsan boyut 2 katına bile çıkmamışken riskin 12 kat azalır.

<img src = "https://i.hizliresim.com/dbmrp21.png">